In [1]:
import time
from PIL import Image
from io import BytesIO
import base64
import os
import psutil

# Required Selenium Modules
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [2]:
### All the parameters needed for the bot
### Once browser launches, log in to Bumble manually
SCREENSHOTS_DIR = os.path.join(os.getcwd(), 'hinge_bot', 'data', 'bumble_screenshots')
CLICK_FAILURE_THRESHOLD = 10
WAIT_TIME = 2

# Browser options
headless = False
options = Options()
if headless:
    # Experimental option to not show browser
    options.add_argument("headless")
options.add_argument('disable-notifications')
options.add_argument('no-sandbox')
options.add_argument("start-maximized")
options.add_argument("inprivate")

# Launch browser
browser = webdriver.Edge(options = options)
browser.get('https://www.bumble.com/')

# Function for getting first profile pic
def get_bumble_screenshot(browser, screenshots_dir):
    latest_sc_num = max([int(x.split('.')[0].split('_')[-1]) for x in os.listdir(screenshots_dir)])
    photo_element = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "album__story")]')))
    photo_element = photo_element.find_element(By.TAG_NAME, 'picture').find_element(By.TAG_NAME, 'img')
    photo_location = photo_element.location
    photo_size = photo_element.size
    photo_screenshot = browser.get_screenshot_as_base64().encode()
    screenshot = Image.open(BytesIO(base64.b64decode(photo_screenshot)))
    screenshot = screenshot.crop((photo_location['x'], photo_location['y'], photo_location['x'] + photo_size['width'], photo_location['y'] + photo_size['height']))
    screenshot.save(os.path.join(screenshots_dir, f'screenshot_{latest_sc_num + 1}.png'))
    return

In [3]:
### Looping through profiles
NUM_PROFILES = 1000
failures = 0
for i in range(NUM_PROFILES):
    if psutil.Process(browser.service.process.pid).is_running() == False:
        break
    try:
        like_button = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, "//div[(@role='button') and (@aria-label = 'Like')]")))
        get_bumble_screenshot(browser, SCREENSHOTS_DIR)
        like_button.click()
    except:
        failures += 1
    
    if failures > CLICK_FAILURE_THRESHOLD:
        browser.close()
        break

    time.sleep(WAIT_TIME)